In [98]:
import csv
import pprint
import re

from os import listdir

import pdfplumber

HEADERS = (
    'work_order',
    'service_code',
    'service_address',
    'latitude',
    'longitude',
    'an_action_date',
    'an_status',
    'an_action_by',
    'an_action_notes',
    'rs_activity',
    'rs_type',
    'rs_resource',
    'rs_qty',
    'rs_units',
    'rs_performed_on',
    'images',
)

# get the file name list
pdf_files = listdir('pdf')
# for dev, just first three
pdf_files = pdf_files[:25]
# print(pdf_files)

masterlist = []

for file_name in pdf_files:
    
    with pdfplumber.open('pdf/{}'.format(file_name)) as pdf:
        
        pages = pdf.pages

        for page in pages:

            lines = page.extract_text()
            
            # check for lines as not all pages have them
            # sometimes last page is just embedded images
            if lines:

                lines = enumerate(lines.split('\n'))
                iter_lines = iter(lines)

                for line_id, line in iter_lines:
                    # get Work Order ID
                    if line.startswith('Work Order ID'):
                        work_order_id_plus = line.split('Work Order ID: ')[1]
                        work_order_id = work_order_id_plus.split(' Work Order Date: ')[0]
            #             print(work_order_id)

                    # get Service Code
                    if line.startswith('Service Code'):
                        service_code, = re.findall(r'\[([^\]]*)\]', line)

                    # get Service Address
                    if line.startswith('Service Address'):
                        index, service_address = next(iter_lines)

                    # get Latitude, Longitude
                    if line.startswith('Latitude'):
                        index, lat_lng_string = next(iter_lines)
                        lat, lng = lat_lng_string.split()

        #             print(line_id, line) # to see top lines

            # get the Action Notes & Result Summary tables
            tables = page.extract_tables()
            print(page, len(tables))
            
            for table in tables:
#                 pprint.pprint(table)
                # get the Action Notes rows
                if table[0][0] == 'Action Date':
                    # skip first row, it's headers
                    for row in table[1:]:
                        print(row)
                        masterlist.append([int(work_order_id), service_code, service_address.strip(), float(lat), float(lng)] + row + ["","","","","","",""])

                
#             print(">>> Found {} tables in {} on {}".format(len(page.extract_tables()) , file_name, page))
#             print(">>> BAD PDF: {}, '{}', '{}', {} {}".format(work_order_id, service_code, service_address, lat, lng), '\n\n')

# turning off because it was just adding an empty row
#         masterlist.append([int(work_order_id), service_code, service_address, float(lat), float(lng),"","","","","","","","","","",""])
        print('{}, "{}", "{}", {} {}'.format(work_order_id, service_code, service_address, lat, lng))

with open("working.csv", "w") as outfile:
    writer = csv.writer(outfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
    writer.writerow(HEADERS)
    for row in masterlist:
        writer.writerow(row)
    
#     im = p0.to_image()
#     im.debug_tablefinder()
#     im.save('/Users/jpheasly/Development/homeless-cleanups/foo.png', format="PNG")
    

<Page:1> 3
['10/26/20  8:04:48 AM', 'Assigned', 'McHarry, Kelsey', 'Work Order Created']
['10/27/20  4:14:48 PM', 'Closed', 'McHarry, Kelsey', 'Automtically Closed by\nSystem.']
['10/27/20  4:14:49 PM', 'Complete', 'McHarry, Kelsey', 'Camp gone trash removed\nWork Order completed by\nKelsey McHarry.']
<Page:2> 0
348010, "PFI10", "SORREL POND", 44.0638111163713 -123.0722084806
<Page:1> 2
['05/01/20 12:05:00 PM', 'Assigned', 'ValladolidSalas, Ricardo']
['05/01/20 12:05:00 PM', 'Assigned', 'ValladolidSalas, Ricardo']
['05/04/20 12:36:59 PM', 'In Process', 'ValladolidSalas, Ricardo']
['05/04/20 12:40:48 PM', 'Closed', 'ValladolidSalas, Ricardo']
['06/23/20  9:17:03 AM', 'Complete', 'Spangler, Mike']
<Page:2> 1
<Page:3> 2
<Page:4> 0
321918, "SFS30", "VAN BUREN ST/W 7TH AVE", 44.0512346753638 -123.109441828038
<Page:1> 3
['09/24/20 10:21:33 AM', 'Assigned', 'Waksmundski, Joe', 'Work Order Created']
['09/29/20  2:45:03 PM', 'Closed', 'Waksmundski, Joe', 'Automtically Closed by\nSystem.']
['09